# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [258]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [259]:
# your code here
orders = pd.read_csv('Orders.csv')
orders.head()


,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [260]:
amount_spent = orders.groupby(by = ['CustomerID', 'Country']).agg({'amount_spent':sum})
amount_spent

condition = amount_spent['amount_spent']>40000
amount_spent[condition]

,,amount_spent
CustomerID,Country,
12346,United Kingdom,77183.60
12415,Australia,124914.53
12931,United Kingdom,42055.96
13089,United Kingdom,58825.83
13694,United Kingdom,65039.62
14088,United Kingdom,50491.81
14096,United Kingdom,65164.79
14156,EIRE,117379.63
14298,United Kingdom,51527.30


In [261]:
amount_spent.quantile()

amount_spent    671.96
Name: 0.5, dtype: float64

In [262]:
labels = ["VIP", "Preferred", " Moderate", "Low"]

qbins = pd.qcut(amount_spent['amount_spent'],4, labels = labels)

qbins.value_counts()

Low          1087
Preferred    1087
VIP          1087
 Moderate    1086
Name: amount_spent, dtype: int64

In [263]:
qbins

CustomerID  Country       
12346       United Kingdom          Low
12347       Iceland                 Low
12348       Finland                 Low
12349       Italy                   Low
12350       Norway            Preferred
                                ...    
18280       United Kingdom          VIP
18281       United Kingdom          VIP
18282       United Kingdom          VIP
18283       United Kingdom          Low
18287       United Kingdom          Low
Name: amount_spent, Length: 4347, dtype: category
Categories (4, object): [VIP < Preferred < Moderate < Low]

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [264]:
# your code here

x = orders.merge(qbins, on='CustomerID')

y = x.groupby(by = ['Country', 'amount_spent_y']).agg({'amount_spent_y':'count'})

y['status'] = y.index.get_level_values(1)

In [265]:
condition = y['status']=="VIP" 
    
maximum = y[condition]['amount_spent_y'].max()

another_condition = y['amount_spent_y'] == maximum

y[another_condition]

,,amount_spent_y,status
Country,amount_spent_y,,
United Kingdom,VIP,18066,VIP


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [266]:
# your code here
y

amount_spent_y     status
Country        amount_spent_y                           
Australia      VIP                          0        VIP
               Preferred                   64  Preferred
                Moderate                  296   Moderate
               Low                       1028        Low
Austria        VIP                         12        VIP
...                                       ...        ...
United Kingdom Low                     221900        Low
Unspecified    VIP                          0        VIP
               Preferred                  188  Preferred
                Moderate                   56   Moderate
               Low                          0        Low

[148 rows x 2 columns]

In [267]:
status = ['VIP' , 'Preferred']
filtered = y['status'].isin(status)

condition = y.loc[filtered]

#condition

z = condition.groupby(by = ['Country']).agg({'amount_spent_y':'sum'})

sort = z.sort_values(by=['amount_spent_y'] , ascending=False)
sort.head(1)

,amount_spent_y
Country,
United Kingdom,55122
